<a href="https://colab.research.google.com/github/AV-Gorshkov/Python_netology/blob/main/PY_%D0%9A%D0%BE%D0%BB%D0%BB%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%B0%D1%8F_%D1%84%D0%B8%D0%BB%D1%8C%D1%82%D1%80%D0%B0%D1%86%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Задание***

 датасет MovieLens:
  - использовать пакет SURPRISE (любые модели из пакета)
  - получите RMSE на тестовом сете 0,87 и ниже.

In [ ]:
# загрузка библиотеки surprise

!pip install surprise

In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook
from scipy.spatial.distance import cityblock, cosine, euclidean, hamming, jaccard

from surprise import KNNWithMeans, KNNBaseline,  KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#  загрузка данных
param = r'/content/drive/My Drive/Нетология/ML/Рекомендательные системы/'

movies = pd.read_csv(f'{param}movies.csv')
ratings = pd.read_csv(f'{param}ratings.csv')

movies.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# объединим таблицы с рейтингами и названиями фильмов

movies_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_ratings.dropna(inplace=True)

movies_ratings.head(5)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
#  посмотрим на размер датасета  (кол-во уникальных пользователей и фильмов)

print(f"фильмы: {len(movies_ratings['movieId'].unique())}.")
print(f"пользователи: {len(movies_ratings['userId'].unique())}.")

фильмы: 9724.
пользователи: 610.


In [ ]:
# датасет для Surprise

dataset = pd.DataFrame( {
    'u_id'   : movies_ratings['userId'],
    'i_id'   : movies_ratings['movieId'],
    'rating' : movies_ratings['rating']
})

dataset.head()

,u_id,i_id,rating
0,1,1,4.0
1,5,1,4.0
2,7,1,4.5
3,15,1,2.5
4,17,1,4.5


In [ ]:
#  Max и Min значения по рейтингу
print( dataset['rating'].min(), dataset['rating'].max() )

0.5 5.0


In [ ]:
# данные для создания модели

reader = Reader(rating_scale=(dataset['rating'].min(), dataset['rating'].max()))
data = Dataset.load_from_df(dataset, reader)

In [ ]:
# создадим таблицу для записи результатов
# Метрики качества: | модель | подход | RMSE |
total = pd.DataFrame( columns=['модель','подход', 'RMSE'])
total

,модель,подход,RMSE


In [ ]:
# модель KNNWithMeans на основе кроссвалидации
# user_based подход

algo = KNNWithMeans (k=40, sim_options={'name': 'pearson_baseline', 'user_based': True})

res = cross_validate(algo, data, measures=['RMSE'], cv=50)

print(f"RMSE: {np.mean(res['test_rmse'])}.")

total = pd.concat( [total, pd.DataFrame([{ 'модель': 'KNNWithMeans','подход': "user_based", 'RMSE': np.mean(res['test_rmse']) } ])\
                     ] )

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [ ]:
#  item based подход

algo = KNNWithMeans(k=40, sim_options={'name': 'pearson_baseline', 'user_based': False})
result = cross_validate(algo, data, measures=['RMSE'], cv=50)
print(f"RMSE: {np.mean(result['test_rmse'])}.")

total = pd.concat( [total, pd.DataFrame([{ 'модель': 'KNNWithMeans','подход': "item_based", 'RMSE': np.mean(res['test_rmse']) } ])\
                     ] )

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

Результат модели KNNWithMeans:

user_based подход  RMSE = 0.881

item based подход  RMSE = 0.867

In [ ]:
# модель KNNBaseline.
# user_based подход

algo = KNNBaseline(k=40, sim_options={'name': 'pearson_baseline', 'user_based': True})
res = cross_validate(algo, data, measures=['RMSE'], cv=50)

print(f"RMSE: {np.mean(res['test_rmse'])}.")
total = pd.concat( [total, pd.DataFrame([{ 'модель': 'KNNBaseline','подход': "user_based", 'RMSE': np.mean(res['test_rmse']) } ])\
                     ] )

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [ ]:
# item based подход

algo = KNNBaseline(k=40, sim_options={'name': 'pearson_baseline', 'user_based': False})
res = cross_validate(algo, data, measures=['RMSE'], cv=50)

print(f"RMSE: {np.mean(res['test_rmse'])}.")
total = pd.concat( [total, pd.DataFrame([{ 'модель': 'KNNBaseline','подход': "item_based", 'RMSE': np.mean(res['test_rmse']) } ])\
                     ] )

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

Результат модели KNNBaseline:

user_based подход  RMSE = 0.863

item based подход  RMSE = 0.841

In [ ]:
#  модель SVD
from surprise import SVD

# разделим выборку
# trainset, testset = train_test_split(data, test_size=.15, random_state=42)

algo = SVD(n_factors=40, n_epochs=30)
# algo.fit(trainset)
# test_pred = algo.test(testset)
# accuracy.rmse(test_pred, verbose=True)

res = cross_validate(algo, data, measures=['RMSE'], cv=50)

print(f"RMSE: {np.mean(res['test_rmse'])}.")
total = pd.concat( [total, pd.DataFrame([{ 'модель': 'SVD','подход': "-", 'RMSE': np.mean(res['test_rmse']) } ])\
                     ] )

RMSE: 0.861551528528914.


In [ ]:
total.sort_values(by = 'RMSE', ascending = True, inplace=True)

total

,модель,подход,RMSE
0,KNNBaseline,item_based,0.841299
0,SVD,-,0.861552
0,KNNBaseline,user_based,0.863023
0,KNNWithMeans,user_based,0.880957
0,KNNWithMeans,item_based,0.880957


лучшая модель KNNBaseline (item based подход), RMSE = 0,841